# Sentence to PClean query demo
This notebook demos the sentence-to-PClean pipeline. Given a sentence giving information about some doctor, it generates and runs queries about those people against the Medicare dataset used in Alex Lew's paper using a PClean model trained by Ian Limarta.


In [34]:
import gc
import json
import logging
from pathlib import Path
import sqlite3
import string
from time import sleep
from typing import Any, Optional, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
from tqdm import tqdm
import vllm

import genparse
from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [8]:
debug_sentences_path = Path().resolve() / "debug_sentences.jsonl"
debug_sentences = [json.loads(line) for line in debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

In [9]:
detailed_debug_sentences_path = Path().resolve() / "debug_sentences_detailed.jsonl"
detailed_debug_sentences = [json.loads(line) for line in detailed_debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

## Example inputs and outputs

Taken from the first ten sentences (rows 2-11) in the [GPT-4 tweets Marjorie Freedman generated][gpt4_tweets].

[gpt4_tweets]: https://docs.google.com/spreadsheets/d/1vq_HAdbFY079vYVOppkFLk_zojJ46MJKsy-TzUm7Q3M/edit

In [149]:
sentence_table_row_template = string.Template("| $sentence | <pre>$example_output</pre> |")
def show_sentences(sentence_data):
    lines = ["| Sentence | Example PClean preamble code |", "| --- | --- |"]
    for sentence_datum in sentence_data:
        try:
            lines.append(sentence_table_row_template.substitute(sentence=sentence_datum["sentence"], example_output=sentence_datum["example_output"].replace("\n", "<br>")))
        except KeyError as e:
            logger.debug('Oops, this example is missing a key... `%s` lacks key `%s`', sentence_datum, e)
    markdown = "\n".join(lines)
    display(Markdown(markdown))


show_sentences(debug_sentences)


2024-07-31 21:45:24 - DEBUG - notebook -   Oops, this example is missing a key... `{'sentence': 'Dr. Kilmore in Boston saved my life with his incredible cardiology skills. Forever grateful! #BostonDocs #HeartHealth'}` lacks key `'example_output'`
2024-07-31 21:45:24 - DEBUG - notebook -   Oops, this example is missing a key... `{'sentence': 'Just had a life-changing neurology consult with Dr. Faraday in Seattle. Truly a genius! #SeattleScience #NeuroExperts'}` lacks key `'example_output'`
2024-07-31 21:45:24 - DEBUG - notebook -   Oops, this example is missing a key... `{'sentence': 'My skin has never been better thanks to Dr. Linwood in Denver. Highly recommend! #DenverDerm #SkinCare'}` lacks key `'example_output'`
2024-07-31 21:45:24 - DEBUG - notebook -   Oops, this example is missing a key... `{'sentence': 'Dr. Voss in Miami was amazing with my child’s treatment. Pediatric care at its finest! #MiamiMeds #KidsHealth'}` lacks key `'example_output'`
2024-07-31 21:45:24 - DEBUG - noteb

| Sentence | Example PClean preamble code |
| --- | --- |
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "neurologist"</pre> |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>address_key = PClean.resolve_dot_expression(trace.model, :Obs, :address)<br>row_trace[address_key] = "321 Pine St."</pre> |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "cardiologist"</pre> |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"</pre> |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>name_key = PClean.resolve_dot_expression(trace.model, :Obs, :name)<br>row_trace[name_key] = "Kay Ryan"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :city)<br>row_trace[city_key] = "Baltimore"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "pediatrician"</pre> |
| Loved visiting Dr. Kilmore's obstetrics practice in Camp Hill! Can't say enough good things. | <pre>last_key = PClean.resolve_dot_expression(trace.model, :Obs, :(p.last))<br>row_trace[last_key] = "Kilmore"<br>city_key = PClean.resolve_dot_expression(trace.model, :Obs, :(b.city_name))<br>row_trace[city_key] = "Camp Hill"<br>occupation_key = PClean.resolve_dot_expression(trace.model, :Obs, :occupation)<br>row_trace[occupation_key] = "obstetrics"</pre> |

## Helper code

### SpaCy code

In [11]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = 'en_core_web_trf'
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = 'PERSON'
_LOCATION_LABEL = 'LOC'
# spaCy labels list example thanks to Stack Overflow user 'russhoppa': https://stackoverflow.com/a/78252807
_SPACY_LABELS = spacy_model.get_pipe("ner").labels
assert _PERSON_LABEL in _SPACY_LABELS

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

def get_locations(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _LOCATION_LABEL]
    )

def show_ents(sentence: str) -> None:
    print(sentence)
    print(spacy_model(sentence).ents)

for sentence_datum in debug_sentences:
    show_ents(sentence_datum["sentence"])
    print()

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
(Kay Ryan, Baltimore, Baltimore)

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
(Kay Ryan, Baltimore, CityCare)

Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
(Baltimore, Kay Ryan)

Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
(Kay Ryan, today)

Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
(Kay Ryan, Baltimore)

Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
(Kay Ryan, Baltimore)

From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #Medical

### Extracting the generated code from a response

In [12]:
def extract_code_from_response(text: str) -> str:
    result = text.removeprefix('<|start_header_id|>assistant<|end_header_id|>').strip()
    return result

### Sort posterior

In [13]:
def sort_posterior(posterior):
    return {inference: likelihood for inference, likelihood in sorted(posterior.items(), key=lambda t: (t[1], t[0]), reverse=True)}

### Aggregate likelihoods over extracted code

In [14]:
def get_aggregate_likelihoods(posterior: dict[str, float]) -> dict[str, float]:
    result = {}
    for inference, likelihood in posterior.items():
        code_only = extract_code_from_response(inference)
        try:
            code_only = json.dumps(
                {key: value for key, value in sorted(json.loads(code_only).items(), key=lambda t: t[0])},
                indent=2
            )
        except json.JSONDecodeError:
            pass
        result.setdefault(code_only, 0.0)
        result[code_only] += likelihood
    return sort_posterior(result)

### Get best inference

In [15]:
def get_best_inference(posterior: dict[str, float]) -> tuple[str, float]:
    return max(posterior.items(), key=lambda t: (t[1], t[0]))

### Running inference locally

In [16]:
SERVER_MODEL_ID = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
SERVER_GENPARSE_MODEL_NAME = 'llama3.1'
_LOCAL_BATCH_SIZE = 1

class FakeResponse:
    def __init__(self, data):
        self._data = data
    
    @property
    def status_code(self):
        return 200

    def json(self):
        return self._data

    def text(self):
        return json.dumps(self._data)

def server_model(grammar, proposal="character"):
    return InferenceSetupVLLM(SERVER_GENPARSE_MODEL_NAME, grammar, proposal_name=proposal)

def run_inference_vllm(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _LOCAL_BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = "",
    inference_setup: InferenceSetupVLLM,
) -> requests.Response:
    #assert batch_size == inference_setup.sampler.llm._model.batch_size
    #assert grammar == inference_setup.sampler.grammar
    #assert (proposal == 'character' and isinstance(inference_setup, genparse.CharacterProposal))
    #assert inference_setup.sampler.temperature == temperature
    # params = vllm.SamplingParams(temperature=1.0, max_tokens=128, n=25)
    # inference = inference_setup(prompt, n_particles=n_particles, sampling_params=params, max_tokens=max_tokens)
    inference = inference_setup(prompt, n_particles=n_particles, max_tokens=max_tokens)
    return FakeResponse({"log_ml": None, "posterior": inference.posterior})

### Running the server

In [17]:
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-16
_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI = 'http://34.122.30.137:9999/restart'
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'
#_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI = 'http://34.70.201.1:9999/restart'
#_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.70.201.1:8888/infer'

def restart_inference_server():
    basic = requests.auth.HTTPBasicAuth(input("User: "), input('Password: '))
    result = requests.post(_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI, auth=basic)
    return result

def run_inference_server(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> requests.Response:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

In [18]:
#restart_inference_server()

### JSON generation prompt

In [134]:
json_prompt = string.Template(
    """Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. If the sentence calls the doctor "Dr. John Smith," include both the given name (as `"first"`) and surname (as `"last"`). If the sentence calls the doctor "Dr. Smith," include only the surname (as `"last"`). Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: What a great experience with Dr. Williams's pediatrics office in Victoria! My child has never been happier to be jabbed with a needle. #ParentingVictories #Victoria
Output: {"last": "Williams", "specialty": "pediatrics", "city_name": "Victoria"}

Input: John Smith's neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (Camp Hill) is terrible!
Output: {"first": "John", "last": "Smith", "specialty": "neurology", "legal_name": "Happy Brain Services LLC", "addr": "512 Example Street", "city_name": "Camp Hill"}

Input: Loved visiting Dr. Kay Ryan's neurology office (Green Medicine Inc.) at 256 Overflow St (ZIP 17011-2202)! No wait time at all. #CampHill
Output: {"first": "Kay", "last": "Ryan", "legal_name": "Green Medicine Inc", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}

Input: Dr. Pat Rogers's office screwed us! So much for Soulful Medical Services Inc. Took our money and Pat gave us three minutes tops. #BaltimoreSucks
Output: {"first": "Pat", "last": "Rogers", "legal_name": "Soulful Medical Services Inc", "city_name": "Baltimore"}

Input: Dr. Maynard's really rules! They figured out my illness when no one else could. Loving the care here in #Lexington
Output: {"last": "Maynard", "city_name": "Lexington"}

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt(sentence: str) -> str:
    return json_prompt.substitute(sentence=sentence)

print(json_prompt)

In [74]:
json_prompt2 = string.Template(
    """Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (a person's given name, e.g. Susan, Jason, Mark, Jane)
- "last" (a person's surname, e.g. Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: John Smith's neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (Camp Hill) is terrible!
Output: ```json
{"first": "John", "last": "Smith", "specialty": "neurology", "legal_name": "Happy Brain Services LLC", "addr": "512 Example Street", "city_name": "Camp Hill"}
```

Input: Loved visiting Dr. Kay Ryan's zneurology office (Green Medicine Inc.) at 256 Overflow St (ZIP 17011-2202)! No wait time at all. #CampHill
Output: ```json
{"first": "Kay", "last": "Ryan", "legal_name": "Green Medicine Inc", "specialty": "neurology", "addr": "256 Overflow St", "zip": "170112202", "city_name": "Camp Hill"}
```

Input: Dr. Pat Rogers's office screwed us! So much for Soulful Medical Services Inc. Took our money and Pat gave us three minutes tops. #BaltimoreSucks
Output: ```json
{"first": "Pat", "last": "Rogers", "legal_name": "Soulful Medical Services Inc", "city_name": "Baltimore"}
```

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt2(sentence: str) -> str:
    return json_prompt2.substitute(sentence=sentence)

print(json_prompt2)

### JSON grammar

In [138]:
json_grammar = r"""
start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? WS? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
json: "{" NL? OPTIONAL_WS kv_pair ("," WS? NL? OPTIONAL_WS kv_pair)* WS? NL? OPTIONAL_WS "}"
kv_pair: COLUMN OPTIONAL_WS ":" OPTIONAL_WS (STRING | "null")

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city_name" | "zip" | "legal_name") "\""
"""

In [144]:
print(json_grammar)


start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? WS? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
json: "{" NL? OPTIONAL_WS kv_pair ("," WS? NL? OPTIONAL_WS kv_pair)* WS? NL? OPTIONAL_WS "}"
kv_pair: COLUMN OPTIONAL_WS ":" OPTIONAL_WS (STRING | "null")

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\n"
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city_name" | "zip" | "legal_name") "\""



In [132]:
json_grammar2 = """
start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? WS? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
#json: "{" NL? OPTIONAL_WS kv_pair ("," WS? NL? OPTIONAL_WS kv_pair) ~ 0..7 NL? OPTIONAL_WS "}"
json: "{" NL? OPTIONAL_WS kv_pair ("," WS? NL? OPTIONAL_WS kv_pair)* NL? OPTIONAL_WS "}"
kv_pair: COLUMN OPTIONAL_WS ":" OPTIONAL_WS value
value: STRING | NULL

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\\n"
NULL: "null"
#_STRING_CHAR: "a".."z" | "A".."Z" | "0".."9" | "-" | "." | "," | "'" | "&" | " "
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
#STRING: "\\"" _STRING_CHAR ~ 0..60 "\\""
COLUMN: "\\"" ("first" | "last" | "addr" | "specialty" | "city_name" | "zip" | "legal_name") "\\""
"""

In [125]:
print(json_grammar2)


start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
#json: "{" NL? OPTIONAL_WS kv_pair ("," WS? NL? OPTIONAL_WS kv_pair) ~ 0..7 NL? OPTIONAL_WS "}"
json: "{" NL? OPTIONAL_WS kv_pair ("," WS? NL? OPTIONAL_WS kv_pair)* NL? OPTIONAL_WS "}"
kv_pair: COLUMN OPTIONAL_WS ":" OPTIONAL_WS value
value: STRING | NULL

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\n"
NULL: "null"
#_STRING_CHAR: "a".."z" | "A".."Z" | "0".."9" | "-" | "." | "," | "'" | "&" | " "
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
#STRING: "\"" _STRING_CHAR ~ 0..60 "\""
COLUMN: "\"" ("first" | "last" | "addr" | "specialty" | "city_name" | "zip" | "legal_name") "\""



## Debugging

In [79]:
model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

In [80]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

2024-07-31 20:51:08 - DEBUG - urllib3.connectionpool -   Resetting dropped connection: huggingface.co
2024-07-31 20:51:08 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0


In [81]:
def md_html_output_text_for_json(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

### Running the LM unconstrained

In [64]:
try:
    del model
    gc.collect()
except NameError:
    pass

In [67]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM(model_id, dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

2024-07-31 20:36:35 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json HTTP/11" 200 0


INFO 07-31 20:36:35 config.py:1417] Upcasting torch.bfloat16 to torch.float32.
INFO 07-31 20:36:35 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)


2024-07-31 20:36:35 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0
2024-07-31 20:36:35 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/generation_config.json HTTP/11" 200 0


INFO 07-31 20:36:35 model_runner.py:680] Starting to load model meta-llama/Meta-Llama-3.1-8B-Instruct...
INFO 07-31 20:36:36 weight_utils.py:223] Using model weights format ['*.safetensors']


2024-07-31 20:36:36 - DEBUG - filelock -   Attempting to acquire lock 140193929485216 on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:36:36 - DEBUG - filelock -   Lock 140193929485216 acquired on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:36:36 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "GET /api/models/meta-llama/Meta-Llama-3.1-8B-Instruct/revision/main HTTP/11" 200 23976
2024-07-31 20:36:36 - DEBUG - filelock -   Attempting to release lock 140193929485216 on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:36:36 - DEBUG - filelock -   Lock 140193929485216 released on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:36:36 - DEBUG - file

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-31 20:36:45 model_runner.py:692] Loading model weights took 29.9151 GB
INFO 07-31 20:36:46 gpu_executor.py:102] # GPU blocks: 854, # CPU blocks: 1024
INFO 07-31 20:36:46 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-31 20:36:46 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-31 20:37:07 model_runner.py:1181] Graph capturing finished in 21 secs.


2024-07-31 20:37:08 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 20:47:08 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 20:57:08 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 21:07:09 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 21:17:09 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 21:27:09 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 21:37:09 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None


In [135]:
sentence = debug_sentences[-1]["sentence"]
print(sentence)
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=128, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Generated Query {i}: ```{output.text}```')

Dr. Garrison in Phoenix fixed my indigestion and it only took one visit. Best gastroenterologist around! #PhoenixHealth #DigestiveCare
Prompt (745 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- 

Processed prompts: 100%|███| 1/1 [00:01<00:00,  1.68s/it, est. speed input: 442.94 toks/s, output: 516.06 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Garrison",
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Garrison",
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"

In [136]:
sentence_to_outputs = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=128, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    sentence_to_outputs[sentence] = []
    for i, output in enumerate(response.outputs, start=1):
        sentence_to_outputs[sentence].append(output)
        print(f'Generated Query {i}: ```{output.text}```')

2024-07-31 21:23:40 - DEBUG - notebook -   Prompt (741 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)

Please generat

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore


Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.00s/it, est. speed input: 369.94 toks/s, output: 425.85 toks/s]
2024-07-31 21:23:42 - DEBUG - notebook -   Prompt (741 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of t

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}```
Generated Query 8: ```<|sta

Processed prompts: 100%|███| 1/1 [00:02<00:00,  2.21s/it, est. speed input: 335.44 toks/s, output: 468.97 toks/s]
2024-07-31 21:23:45 - DEBUG - notebook -   Prompt (743 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of t

...

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Jamison",
  "last": "Jamison",
  "specialty": "endocrinologist",
  "city_name": "Dallas"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Jamison",
  "last": "Jamison",
  "specialty": "endocrinologist",
  "city_name": "Dallas"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Jamison",
  "last": "Jamison",
  "specialty": "endocrinologist",
  "city_name": "Dallas"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Jamison",
  "last": "Jamison",
  "specialty": "endocrinologist",
  "city_name": "Dallas"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Jamison",
  "last": "Jamison",
  "specialty": "endocrinologist",
  "city_name": "Dallas"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Jamison",
  "specialty": "endocri

Processed prompts: 100%|███| 1/1 [00:01<00:00,  1.63s/it, est. speed input: 453.16 toks/s, output: 544.64 toks/s]
2024-07-31 21:24:19 - DEBUG - notebook -   Prompt (745 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of t

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Thornton",
  "specialty": "ophthalmologist",
  "city_name": "LA"
}```
Generated Query 7: ```<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Thorn

Processed prompts: 100%|███| 1/1 [00:01<00:00,  1.64s/it, est. speed input: 453.99 toks/s, output: 555.75 toks/s]

Generated Query 1: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 2: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 3: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 4: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 5: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 6: ```<|start_header_id|>assistant<|end_header_id|>

{
  "last": "Garrison",
  "specialty": "gastroenterologist",
  "city_name": "Phoenix"
}```
Generated Query 7: ```<|start_header_id|

In [139]:
# confirm all sentences are grammatical
guide = genparse.BoolCFGLM(genparse.lark_interface.LarkStuff(json_grammar).char_cfg(0.99))
for outputs in sentence_to_outputs.values():
    for output in outputs:
        for i in range(len(output.text)):
            if output.text[i] not in guide.p_next(output.text[:i]) and i < len(output.text.rstrip()):
                print(f'In output `{output.text[:i]}`: Output char {output.text[i]!r} (index {i}) is not allowed by grammar! Allowed next: {guide.p_next(output.text[:i])}')
                print(f"Invalid output: ```{output.text}```")
                break

In output `<|start_header_id|>assistant<|end_header_id|>

`: Output char '`' (index 47) is not allowed by grammar! Allowed next: Chart {
  '\n': 1,
  ' ': 1,
  '{': 1,
}
Invalid output: ```<|start_header_id|>assistant<|end_header_id|>

```json
{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}
``````
In output `<|start_header_id|>assistant<|end_header_id|>

{
    `: Output char ' ' (index 53) is not allowed by grammar! Allowed next: Chart {
  '"': 1,
}
Invalid output: ```<|start_header_id|>assistant<|end_header_id|>

{
            "last": "Ryan",
            "first": "Kay", 
            "specialty": "pediatrician", 
            "city_name": "Baltimore"
}```
In output `<|start_header_id|>assistant<|end_header_id|>

{
  "first": "Linwood",
  "last": "Linwood",
  "specialty": "dermatologist `: Output char '(' (index 122) is not allowed by grammar! Allowed next: Chart {
  'k': 1,
  '.': 1,
  ' ': 1,
  '0': 1,
  'w': 1,
  '3': 1,
  '6': 1,
  '5': 1,
  '4': 1,
  'G': 1,
  'q'

In [140]:
table_rows = ['| Sentence | Extracted Code | Raw Response | # Tokens | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:| ---:|']
for sentence, outputs in sentence_to_outputs.items():
    try:
        posterior = {}
        n_outputs = {}
        total_tokens = {}
        for i, output in enumerate(outputs):
            posterior.setdefault(output.text, 0.0)
            posterior[output.text] += 1.0 / len(outputs)
            n_outputs.setdefault(output.text, 0)
            n_outputs[output.text] += 1
            total_tokens.setdefault(output.text, 0)
            total_tokens[output.text] += len(output.token_ids)
        aggregate_likelihoods = sort_posterior(get_aggregate_likelihoods(posterior))
        n_outputs_aggregated = {}
        total_tokens_aggregated = {}
        for output in outputs:
            code = extract_code_from_response(output.text)
            try:
                code = json.dumps({key: value for key, value in sorted(json.loads(code).items())}, indent=2)
            except json.JSONDecodeError:
                pass
            n_outputs_aggregated.setdefault(code, 0)
            n_outputs_aggregated[code] += n_outputs[output.text]
            total_tokens_aggregated.setdefault(code, 0)
            total_tokens_aggregated[code] += total_tokens[output.text]
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| {total_tokens_aggregated[code] / n_outputs_aggregated[code]}'
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError as e:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {e} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

| Sentence | Extracted Code | Raw Response | # Tokens | Aggregate Likelihood (Raw LH) |
|:--- |:--- |:--- | ---:| ---:|
| Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 30.06024096385542| (unk) | 60.00% (60.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "consultation"<br>}</pre> | 39.0| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "consultation"<br>}</pre> | 32.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "addr": "",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "",<br>  "zip": ""<br>}</pre> | 55.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 19.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "consultant"<br>}</pre> | 42.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null,<br>  "zip": null<br>}</pre> | 56.0| (unk) | 4.00% (4.00%) |
| Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 31.636363636363637| (unk) | 20.00% (20.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 39.42857142857143| (unk) | 20.00% (20.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 38.166666666666664| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "CityCare"<br>}</pre> | 42.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "legal_name": "Dr. Kay Ryan's office"<br>}</pre> | 45.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "legal_name": "CityCare"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 32.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "null",<br>  "specialty": "null"<br>}</pre> | 59.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": ""<br>}</pre> | 49.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "CityCare"<br>}</pre> | 52.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "",<br>  "zip": ""<br>}</pre> | 56.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": ""<br>}</pre> | 64.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": ""<br>}</pre> | 49.0| (unk) | 4.00% (4.00%) |
| Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 29.925675675675677| (unk) | 80.00% (80.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 21.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": null<br>}</pre> | 30.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "null"<br>}</pre> | 31.0| (unk) | 4.00% (4.00%) |
|     | <pre>```json<br>{<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "city_name": "Baltimore"<br>}<br>```</pre> | 34.0| (unk) | 4.00% (4.00%) |
| Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "neurologist"<br>}</pre> | 41.63414634146341| (unk) | 88.00% (88.00%) |
|     | <pre>{<br>  "addr": "",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "neurologist",<br>  "zip": ""<br>}</pre> | 63.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "Dr.Kay Ryan's medical business",<br>  "specialty": "neurologist"<br>}</pre> | 59.0| (unk) | 4.00% (4.00%) |
| Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 30.3855421686747| (unk) | 68.00% (68.00%) |
|     | <pre>{<br>  "addr": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null,<br>  "zip": null<br>}</pre> | 59.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 19.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "undefined"<br>}</pre> | 38.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "not provided"<br>}</pre> | 39.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "in"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": ""<br>}</pre> | 39.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "",<br>  "zip": ""<br>}</pre> | 52.0| (unk) | 4.00% (4.00%) |
| Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 39.5| (unk) | 72.00% (72.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 31.8| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "addr": "321 Pine St.",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 36.5| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "addr": "321 Pine St",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null<br>}</pre> | 53.0| (unk) | 4.00% (4.00%) |
| From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #MedicalCare | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "cardiologist"<br>}</pre> | 40.77777777777778| (unk) | 80.00% (80.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "cardiologist"<br>}</pre> | 32.0| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "cardiologist"<br>}</pre> | 46.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "cardiologist",<br>  "zip": ""<br>}</pre> | 60.0| (unk) | 4.00% (4.00%) |
| Just wrapped up my appointment with Dr. Kay Ryan in Baltimore. Her professionalism is top-notch. #HealthJourney | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 29.551724137931036| (unk) | 64.00% (64.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 20.5| (unk) | 16.00% (16.00%) |
|     | <pre>{<br>  "addr": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null,<br>  "zip": null<br>}</pre> | 62.5| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "unknown"<br>}</pre> | 38.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "not specified"<br>}</pre> | 39.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "Undisclosed"<br>}</pre> | 42.0| (unk) | 4.00% (4.00%) |
| Dr. Kay Ryan's office in Baltimore is so efficient and welcoming. A great experience overall! #DoctorVisit #CityCare | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan"<br>}</pre> | 29.672727272727272| (unk) | 44.00% (44.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan"<br>}</pre> | 20.058823529411764| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "last": "Ryan",<br>  "specialty": "null"<br>}</pre> | 28.5| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "none"<br>}</pre> | 38.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null,<br>  "zip": null<br>}</pre> | 56.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": null<br>}</pre> | 50.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": "unavailable",<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "unavailable",<br>  "specialty": "unavailable",<br>  "zip": "unavailable"<br>}</pre> | 64.0| (unk) | 4.00% (4.00%) |
| Can't say enough good things about Dr. Kay Ryan, a pediatrician, in Baltimore. Truly an outstanding doctor! #Grateful | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "specialty": "pediatrician"<br>}</pre> | 41.15873015873016| (unk) | 88.00% (88.00%) |
|     | <pre>{<br>  "first": "Kay", "last": "Ryan", "specialty": "pediatrician", "city_name": "Baltimore"</pre> | 36.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": "",<br>  "specialty": "pediatrician"<br>}</pre> | 46.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "addr": null,<br>  "city_name": "Baltimore",<br>  "first": "Kay",<br>  "last": "Ryan",<br>  "legal_name": null,<br>  "specialty": "pediatrician",<br>  "zip": null<br>}</pre> | 65.0| (unk) | 4.00% (4.00%) |
| Loved visiting Dr. Kilmore's obstetrics practice in Camp Hill! Can't say enough good things. | <pre>{<br>  "city_name": "Camp Hill",<br>  "last": "Kilmore",<br>  "specialty": "obstetrics"<br>}</pre> | 35.15909090909091| (unk) | 48.00% (48.00%) |
|     | <pre>{<br>  "city_name": "Camp Hill",<br>  "first": "Kilmore",<br>  "last": "Kilmore",<br>  "specialty": "obstetrics"<br>}</pre> | 46.205882352941174| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "city_name": "Camp Hill",<br>  "first": "Kilmore",<br>  "last": "Kilmore",<br>  "legal_name": "obstetrics practice"<br>}</pre> | 45.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Camp Hill",<br>  "first": "Kilmore",<br>  "specialty": "obstetrics"<br>}</pre> | 30.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Camp Hill",<br>  "first": "Dr. Kilmore",<br>  "specialty": "obstetrics"<br>}</pre> | 36.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Camp Hill",<br>  "first": "Dr. Kilmore",<br>  "last": "Kilmore",<br>  "specialty": "obstetrics"<br>}</pre> | 45.0| (unk) | 4.00% (4.00%) |
| Dr. Kilmore in Boston saved my life with his incredible cardiology skills. Forever grateful! #BostonDocs #HeartHealth | <pre>{<br>  "city_name": "Boston",<br>  "first": "Kilmore",<br>  "last": "Kilmore",<br>  "specialty": "cardiology"<br>}</pre> | 42.12| (unk) | 48.00% (48.00%) |
|     | <pre>{<br>  "city_name": "Boston",<br>  "last": "Kilmore",<br>  "specialty": "cardiology"<br>}</pre> | 32.95454545454545| (unk) | 48.00% (48.00%) |
|     | <pre>{<br>  "city_name": "Boston",<br>  "first": "Kilmore",<br>  "last": "Kilmore",<br>  "legal_name": "",<br>  "specialty": "cardiology"<br>}</pre> | 48.0| (unk) | 4.00% (4.00%) |
| Just had a life-changing neurology consult with Dr. Faraday in Seattle. Truly a genius! #SeattleScience #NeuroExperts | <pre>{<br>  "city_name": "Seattle",<br>  "first": "Faraday",<br>  "last": "Faraday",<br>  "specialty": "neurology"<br>}</pre> | 40.87671232876713| (unk) | 52.00% (52.00%) |
|     | <pre>{<br>  "city_name": "Seattle",<br>  "last": "Faraday",<br>  "specialty": "neurology"<br>}</pre> | 32.78260869565217| (unk) | 48.00% (48.00%) |
| My skin has never been better thanks to Dr. Linwood in Denver. Highly recommend! #DenverDerm #SkinCare | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood"<br>}</pre> | 31.5| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood",<br>  "specialty": "dermatology"<br>}</pre> | 34.5| (unk) | 24.00% (24.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood"<br>}</pre> | 22.2| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood",<br>  "specialty": "dermatology"<br>}</pre> | 43.4| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "last": "Linwood",<br>  "specialty": "Dermatology",<br>  "city_name": "Denver"<br>}<br> <br>Note: In the input, "Dr. Linwood" implies that "Dermatology" is the specialty, assuming "Dr." is used to denote a medical doctor (in medical contexts) we will just assume anything without such a title is a surname only.</pre> | 87.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "last": "Linwood",<br>  "specialty": "Derm"<br>}</pre> | 34.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood",<br>  "specialty": "dermatologist (implied from #DenverDerm #SkinCare)"<br>}</pre> | 52.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood",<br>  "specialty": "derm"<br>}</pre> | 40.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Denver",<br>  "first": "Linwood",<br>  "last": "Linwood",<br>  "specialty": "Dermatology"<br>}</pre> | 44.0| (unk) | 4.00% (4.00%) |
| Dr. Voss in Miami was amazing with my child’s treatment. Pediatric care at its finest! #MiamiMeds #KidsHealth | <pre>{<br>  "city_name": "Miami",<br>  "last": "Voss",<br>  "specialty": "pediatrics"<br>}</pre> | 30.214285714285715| (unk) | 48.00% (48.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "last": "Voss",<br>  "specialty": "pediatric care"<br>}</pre> | 33.0| (unk) | 24.00% (24.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "last": "Voss",<br>  "specialty": "Pediatric care"<br>}</pre> | 34.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "first": "Voss",<br>  "last": "Voss",<br>  "specialty": "pediatric care"<br>}</pre> | 44.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "first": "Voss", <br>  "last": "Voss", <br>  "last": "Voss",  # I will omit this "last" key - value pair for following suggestion<br>  "specialty": "pediatrics", <br>  "city_name": "Miami"<br>}</pre> | 68.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "first": "Voss",<br>  "last": "",<br>  "specialty": "pediatric care"<br>}</pre> | 32.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Miami",<br>  "first": "Voss",<br>  "last": "Voss",<br>  "specialty": "pediatrics"<br>}</pre> | 34.0| (unk) | 4.00% (4.00%) |
| Dr. Ravenswood in San Francisco fixed my knee like a pro. Orthopedic wizard! #SFOOrtho #BoneHealth | <pre>{<br>  "city_name": "San Francisco",<br>  "last": "Ravenswood",<br>  "specialty": "Orthopedic"<br>}</pre> | 35.67741935483871| (unk) | 44.00% (44.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "last": "Ravenswood",<br>  "specialty": "orthopedic"<br>}</pre> | 35.111111111111114| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": "Ravenswood",<br>  "last": "Ravenswood",<br>  "specialty": "orthopedic"<br>}</pre> | 46.0| (unk) | 8.00% (8.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "last": "Ravenswood",<br>  "specialty": "Orthopedics"<br>}</pre> | 36.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": null,<br>  "last": "Ravenswood",<br>  "specialty": "Orthopedic"<br>}</pre> | 42.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": "Ravenswood",<br>  "specialty": "Orthopedic"<br>}</pre> | 39.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "San Francisco",<br>  "first": "Ravenswood",<br>  "last": "Ravenswood",<br>  "specialty": "Orthopedic"<br>}</pre> | 46.0| (unk) | 4.00% (4.00%) |
| Grateful for Dr. Shapiro in Chicago who guided me through my cancer journey. The best oncologist! #ChicagoCare #FightCancer | <pre>{<br>  "city_name": "Chicago",<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 32.265625| (unk) | 56.00% (56.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Shapiro",<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 41.92307692307692| (unk) | 20.00% (20.00%) |
|     | <pre>{<br>  "first": "Shapiro",<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 33.0| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 25.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Shapiro",<br>  "last": "Shapiro"<br>}</pre> | 33.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Chicago",<br>  "first": "Dr. Shapiro",<br>  "last": "Shapiro",<br>  "specialty": "oncologist"<br>}</pre> | 42.0| (unk) | 4.00% (4.00%) |
| Feeling mentally strong thanks to Dr. Everhart in NYC. Exceptional psychiatrist! #NYCWellness #MentalHealth | <pre>{<br>  "city_name": "NYC",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 31.093023255813954| (unk) | 52.00% (52.00%) |
|     | <pre>{<br>  "city_name": "NYC",<br>  "first": "Everhart",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 42.06| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "first": "Everhart"</pre> | 14.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "NyC",<br>  "first": "Everhart",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 42.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "New York City",<br>  "last": "Everhart",<br>  "specialty": "psychiatrist"<br>}</pre> | 37.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "NYC",<br>  "first": "Everhart",<br>  "last": null,<br>  "specialty": "psychiatrist"<br>}</pre> | 40.0| (unk) | 4.00% (4.00%) |
| Dr. Jamison in Dallas balanced my hormones perfectly. Best endocrinologist ever! #DallasDocs #HormoneHealth | <pre>{<br>  "city_name": "Dallas",<br>  "last": "Jamison",<br>  "specialty": "endocrinologist"<br>}</pre> | 32.52777777777778| (unk) | 48.00% (48.00%) |
|     | <pre>{<br>  "city_name": "Dallas",<br>  "first": "Jamison",<br>  "last": "Jamison",<br>  "specialty": "endocrinologist"<br>}</pre> | 41.142857142857146| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "city_name": "Dallas",<br>  "last": "Jamison"<br>}</pre> | 22.2| (unk) | 12.00% (12.00%) |
|     | <pre>{<br>  "city_name": "Dallas",<br>  "last": "Jamison",<br>  "specialty": "endocrinology"<br>}</pre> | 28.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Dallas",<br>  "first": "Jamison",<br>  "last": "Jamison"<br>}</pre> | 31.0| (unk) | 4.00% (4.00%) |
| Seeing clearly again thanks to Dr. Thornton in LA. Incredible ophthalmologist! #LAVisions #EyeCare | <pre>{<br>  "city_name": "LA",<br>  "last": "Thornton",<br>  "specialty": "ophthalmologist"<br>}</pre> | 31.774193548387096| (unk) | 64.00% (64.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "first": "Thornton",<br>  "last": "Thornton",<br>  "specialty": "ophthalmologist"<br>}</pre> | 43.09090909090909| (unk) | 32.00% (32.00%) |
|     | <pre>{<br>  "city_name": "LA",<br>  "first": " Thornton",<br>  "last": " Thornton",<br>  "specialty": "ophthalmologist"<br>}</pre> | 41.0| (unk) | 4.00% (4.00%) |
| Dr. Garrison in Phoenix fixed my indigestion and it only took one visit. Best gastroenterologist around! #PhoenixHealth #DigestiveCare | <pre>{<br>  "city_name": "Phoenix",<br>  "last": "Garrison",<br>  "specialty": "gastroenterologist"<br>}</pre> | 33.672413793103445| (unk) | 64.00% (64.00%) |
|     | <pre>{<br>  "city_name": "Phoenix",<br>  "first": "Garrison",<br>  "last": "Garrison",<br>  "specialty": "gastroenterologist"<br>}</pre> | 43.08| (unk) | 28.00% (28.00%) |
|     | <pre>{<br>  "city_name": "Phoenix",<br>  "last": "Garrison",<br>  "specialty": "gastroenterology"<br>}</pre> | 34.0| (unk) | 4.00% (4.00%) |
|     | <pre>{<br>  "city_name": "Phoenix",<br>  "first": "Garrison",<br>  "last": "Garrison",<br>  "specialty": "Gastroenterologist"<br>}</pre> | 42.0| (unk) | 4.00% (4.00%) |

In [147]:
# confirm all sentences are grammatical
guide = genparse.BoolCFGLM(genparse.lark_interface.LarkStuff(json_grammar).char_cfg(0.99))
for sentence, outputs in sentence_to_outputs.items():
    print(f"SENTENCE: {sentence}")
    for output in outputs:
        good_output = True
        for i in range(len(output.text)):
            if output.text[i] in guide.p_next(output.text[:i]) or i >= len(output.text.rstrip()):
                continue
            else:
                good_output = False
                break
        if not good_output:
            print(f"BAD OUTPUT: {output.text}")

SENTENCE: Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
SENTENCE: Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
SENTENCE: Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
BAD OUTPUT: <|start_header_id|>assistant<|end_header_id|>

```json
{
  "first": "Kay",
  "last": "Ryan",
  "city_name": "Baltimore"
}
```
SENTENCE: Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
SENTENCE: Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
SENTENCE: Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
SENTENCE: From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered 

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

### Running the model with constraints

In [148]:
sentence = debug_sentences[-1]["sentence"].replace("Garrison", "Smith")
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
restart_response = restart_inference_server()
if restart_response.status_code == 200:
    sleep(45)
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

Prompt (745 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)

Please generate a key-value pair for every such value spe

User:  
Password:  


2024-07-31 21:38:59 - DEBUG - urllib3.connectionpool -   Starting new HTTP connection (1): 34.122.30.137:9999
2024-07-31 21:38:59 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:9999 "POST /restart HTTP/11" 401 90
2024-07-31 21:38:59 - DEBUG - urllib3.connectionpool -   Starting new HTTP connection (1): 34.122.30.137:8888
2024-07-31 21:39:03 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:8888 "POST /infer HTTP/11" 200 1700


Generated Query 1 (likelihood 20.00%): ```<|start_header_id|>assistant<|end_header_id|>
{"first": "Smith", "specialty": "gastroenterology", "city_name": "Phoenix"}```
Generated Query 2 (likelihood 6.67%): ```<|start_header_id|>assistant<|end_header_id|>
{"first": null, "last": "Smith", "specialty": "gastroenterology", "city_name": "Phoenix", "zip": "75040"}```
Generated Query 3 (likelihood 13.33%): ```<|start_header_id|>assistant<|end_header_id|>
{"first": null, "last": "Smith", "specialty": "gastroenterology", "city_name": "Phoenix"}```
Generated Query 4 (likelihood 20.00%): ```<|start_header_id|>assistant<|end_header_id|>
{"last": "Smith", "city_name": "Phoenix", "specialty": "gastorentologist"}```
Generated Query 5 (likelihood 40.00%): ```<|start_header_id|>assistant<|end_header_id|>
{"last": "Smith", "first": "Dr.", "specialty": "gastroenterologist"}```


In [52]:
restart_response = restart_inference_server()


User:  genparser
Password:  earley


2024-07-31 20:31:05 - DEBUG - urllib3.connectionpool -   Starting new HTTP connection (1): 34.122.30.137:9999
2024-07-31 20:31:06 - DEBUG - urllib3.connectionpool -   http://34.122.30.137:9999 "POST /restart HTTP/11" 200 51


In [102]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    print(response)

2024-07-31 21:03:45 - DEBUG - notebook -   Prompt (766 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name, e.g., Susan, Jason, Mark, Jane)
- "last" (the doctor's surname, e.g., Smith, Harris, Newman, Johnson)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)

Please generat

ConnectionError: HTTPConnectionPool(host='34.122.30.137', port=8888): Max retries exceeded with url: /infer (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f817945ded0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
table_rows = ['| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        aggregate_likelihoods = get_aggregate_likelihoods(posterior)
        # for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
        #     # Assert the response parses properly
        #     pclean_parser.parse(output_text)

        #     code = extract_code_from_response(output_text)
        #     aggregate_likelihood = aggregate_likelihoods[code]
        #     code_with_br = code.replace("\n", "<br>")
        #     output_with_br = md_html_output_text(output_text)
        #     table_rows.append(
        #         f'| {sentence if i == 1 else "   "} '
        #         f'| <pre>{code_with_br}</pre> '
        #         f'| <pre>{output_with_br}</pre> '
        #         f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
        #     )
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {response.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

### Local inference

In [66]:
try:
    del inference_setup
    gc.collect()
except NameError:
    pass

In [59]:
try:
    del inference_setup
    gc.collect()
except NameError:
    pass

#inference_setup = server_model(json_grammar)
inference_setup = genparse.backends.vllm.InferenceSetupVLLM('llama3.1', json_grammar, proposal_name='character')

2024-07-31 20:33:11 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json HTTP/11" 200 0


INFO 07-31 20:33:11 config.py:68] Updating rope_scaling from {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'} to {'type': 'dynamic', 'factor': 1.0}
INFO 07-31 20:33:11 config.py:1417] Upcasting torch.bfloat16 to torch.float32.
INFO 07-31 20:33:11 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling={'type': 'dynamic', 'factor': 1.0}, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float32, max_seq_len=7760, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config

2024-07-31 20:33:11 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0
2024-07-31 20:33:11 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/generation_config.json HTTP/11" 200 0


INFO 07-31 20:33:11 model_runner.py:680] Starting to load model meta-llama/Meta-Llama-3.1-8B-Instruct...
INFO 07-31 20:33:12 weight_utils.py:223] Using model weights format ['*.safetensors']


2024-07-31 20:33:12 - DEBUG - filelock -   Attempting to acquire lock 140193920493568 on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:33:12 - DEBUG - filelock -   Lock 140193920493568 acquired on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:33:12 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "GET /api/models/meta-llama/Meta-Llama-3.1-8B-Instruct/revision/main HTTP/11" 200 23976
2024-07-31 20:33:12 - DEBUG - filelock -   Attempting to release lock 140193920493568 on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:33:12 - DEBUG - filelock -   Lock 140193920493568 released on /var/tmp/1e06ffef4faccfa02542c605236a96ba8ec984243111d7be1d4b1409c032448emeta-llama-Meta-Llama-3.1-8B-Instruct.lock
2024-07-31 20:33:12 - DEBUG - file

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 07-31 20:33:21 model_runner.py:692] Loading model weights took 29.9151 GB
INFO 07-31 20:33:22 gpu_executor.py:102] # GPU blocks: 798, # CPU blocks: 1024
INFO 07-31 20:33:22 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-31 20:33:22 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-31 20:33:43 model_runner.py:1181] Graph capturing finished in 21 secs.


2024-07-31 20:33:43 - DEBUG - urllib3.connectionpool -   https://huggingface.co:443 "HEAD /meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/tokenizer_config.json HTTP/11" 200 0
2024-07-31 20:33:44 - DEBUG - urllib3.connectionpool -   Resetting dropped connection: stats.vllm.ai
2024-07-31 20:33:44 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 20:43:45 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 20:53:45 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 21:03:45 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 21:13:45 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 200 None
2024-07-31 21:23:45 - DEBUG - root -   Failed to send usage data to server
2024-07-31 21:33:45 - DEBUG - urllib3.connectionpool -   https://stats.vllm.ai:443 "POST / HTTP/11" 

In [30]:
local_outputs = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    #server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'grammar': json_grammar}
    #response = run_inference_vllm(prompt, inference_setup=inference_setup, **server_inference_params)
    #responses[sentence] = response
    output = inference_setup(prompt=prompt, max_tokens=128, n_particles=15)
    print(output.log_ml_estimate)
    print(output.posterior)
    local_outputs[sentence]

2024-07-31 20:09:27 - DEBUG - notebook -   Prompt (794 tokens): ```<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    “some_feature”: “foo”,
    “another_feature”: “bar”,
    ...
    “last_feature”: “baz”
}
```

There are eight features we would like to extract from the sentence:

- “first” (the doctor’s given name)
- “last” (the doctor’s surname)
- “specialty” (the doctor’s specialty)
- “legal_name” (the legal name of the doctor’s business, practice, or employer)
- “city_name” (the city where the doctor’s office is)
- “zip” (the ZIP code of the doctor’s office)
- “addr” (the doctor’s office address, “line 1" -- this is the street address)
- “addr2” (additional address information giving a suite number, unit number, et cete

TypeError: LLM._validate_and_add_requests() missing 1 required positional argument: 'prompt_adapter_request'

In [ ]:
try:
    del inference_setup
    gc.collect()
except NameError:
    pass

### Running the model on an arbitrary sentence

In [ ]:
responses = {}
for sentence_datum in tqdm([#{"sentence": "John Smith the podiatrist is terrible, so much worse than John Smith the cardiologist."}, {"sentence": "John Smith works with Jack Smith at the neurology office. He’s terrible at neurology."},
                            {"sentence": "John Smith's office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible!"}]):
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    #break

In [ ]:
table_rows = ['| Sentence | Extracted JSON | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
            # Assert the response parses properly
            # TODO grammar no longer parses in lark but does in genparse so idk, can't test this as easily
            #json_parser.parse(output_text)

            #code = extract_code_from_response(output_text)
            code = output_text
            code_with_br = code.replace("\n", "<br>")
            output_with_br = md_html_output_text_for_json(output_text)
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre></pre> '
                f'| <pre>{output_with_br}</pre> '
                f'| {100 * likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {response.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))